<a href="https://colab.research.google.com/github/yeop6134/Ai_humanities/blob/main/BERT%EA%B8%B0%EB%B0%98TopicModeling/Korean_CTM.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

In [ ]:
!pip install contextualized-topic-models==2.2.0
!pip install pyldavis

# Colab에 Mecab 설치
!git clone https://github.com/SOMJANG/Mecab-ko-for-Google-Colab.git
%cd Mecab-ko-for-Google-Colab
!bash install_mecab-ko_on_colab190912.sh

In [ ]:
%cd "/content/drive/MyDrive/Colab Notebooks/인공지능인문학"

/content/drive/MyDrive/Colab Notebooks/인공지능인문학


In [ ]:
# with open("discrimination.txt", "r", encoding='utf-8-sig') as f:
#     text_file = f.readlines()

In [ ]:
from contextualized_topic_models.models.ctm import CombinedTM
from contextualized_topic_models.utils.data_preparation import TopicModelDataPreparation, bert_embeddings_from_list
from contextualized_topic_models.utils.preprocessing import WhiteSpacePreprocessing
from sklearn.feature_extraction.text import CountVectorizer
from konlpy.tag import Mecab
from tqdm import tqdm

In [ ]:
f = open("discrimination.txt")

In [ ]:
f.readline()

'text\n'

In [ ]:
documents = [line.strip() for line in open("discrimination.txt", encoding="utf-8").readlines()]

In [ ]:
documents[:5]

['text',
 '남자들은 도대체 왜 그러는걸까?',
 '말하면서 침 개많이 튀길것 같아.',
 '머리가 멍청해서 못읽는거아니냐?',
 '"동유럽, 러시아 애들은 가난해도 사랑으로 결혼하는데 헬조선은 자본주의의 지옥 그 자체다"']

In [ ]:
class CustomTokenizer:
    def __init__(self, tagger):
        self.tagger = tagger
    def __call__(self, sent):
        word_tokens = self.tagger.morphs(sent)
        result = [word for word in word_tokens if len(word) > 1]
        return result
     

In [ ]:
custom_tokenizer = CustomTokenizer(Mecab())

In [ ]:
vectorizer = CountVectorizer(tokenizer=custom_tokenizer, max_features=3000)

In [ ]:
train_bow_embeddings = vectorizer.fit_transform(documents)

In [ ]:
print(train_bow_embeddings.shape)

(4436, 3000)


In [ ]:
vocab = vectorizer.get_feature_names()
id2token = {k: v for k, v in zip(range(0, len(vocab)), vocab)}

/usr/local/lib/python3.8/dist-packages/sklearn/utils/deprecation.py:87: FutureWarning: Function get_feature_names is deprecated; get_feature_names is deprecated in 1.0 and will be removed in 1.2. Please use get_feature_names_out instead.
  warnings.warn(msg, category=FutureWarning)


In [ ]:
len(vocab)

3000

In [ ]:
train_contextualized_embeddings = bert_embeddings_from_list(documents, \
                                                            "sentence-transformers/xlm-r-100langs-bert-base-nli-stsb-mean-tokens")

Downloading:   0%|          | 0.00/574 [00:00<?, ?B/s]

Downloading:   0%|          | 0.00/190 [00:00<?, ?B/s]

Downloading:   0%|          | 0.00/4.06k [00:00<?, ?B/s]

Downloading:   0%|          | 0.00/731 [00:00<?, ?B/s]

Downloading:   0%|          | 0.00/122 [00:00<?, ?B/s]

...

Downloading:   0%|          | 0.00/150 [00:00<?, ?B/s]

Downloading:   0%|          | 0.00/9.10M [00:00<?, ?B/s]

Downloading:   0%|          | 0.00/527 [00:00<?, ?B/s]

Downloading:   0%|          | 0.00/229 [00:00<?, ?B/s]

Batches:   0%|          | 0/23 [00:00<?, ?it/s]

In [ ]:
qt = TopicModelDataPreparation()

training_dataset = qt.load(train_contextualized_embeddings, train_bow_embeddings, id2token)

# Combined TM 학습하기

In [ ]:
ctm = CombinedTM(bow_size=len(vocab), contextual_size=768, n_components=8, num_epochs=20)
ctm.fit(training_dataset)

Epoch: [20/20]	 Seen Samples: [88720/88720]	Train Loss: 37.582895428560356	Time: 0:00:01.309998: : 20it [00:27,  1.40s/it]


# 토픽들
학습 후에는 토픽 모델이 선정한 토픽들을 보려면 아래의 메소드를 사용합니다.  


> get_topic_lists


해당 메소드에는 각 토픽마다 몇 개의 단어를 보고 싶은지에 해당하는 파라미터를 넣어즐 수 있습니다.

In [ ]:
ctm.get_topics(10)

defaultdict(list,
            {0: ['여자', '이나', '어야', '해야', '나이', '예쁘', '운동', '예쁜', '당연히', '운전'],
             1: ['남자', '아니', '여자', '는데', 'ㅋㅋ', '보다', '니까', '잘못', '그런', '한테'],
             2: ['#@#',
              '찾아보',
              '수저',
              '사기',
              '과학',
              '고향',
              '캠퍼스',
              '공기업',
              '감자',
              '천주교인'],
             3: ['남자', '인간', '#@#', 'ㅋㅋ', '#@', '집단', '인물', '정치인', '여성', '여자'],
             4: ['순간',
              '주제',
              '잔말',
              'ㅠㅠ',
              '당신',
              '보단',
              '비해',
              '깜둥이',
              '쳐다봐서',
              '자네'],
             5: ['해라',
              '한잔',
              '의경',
              '커피',
              '도서관',
              '차린',
              '쳐다봐서',
              '그건',
              '잔말',
              '그래'],
             6: ['장애', '공부', '부모', '다니', '학교', '가난', '자식', '면서', '해서', '에서'],
             7: ['중국',
              '중국인',

- 0번 토픽은 '여자', '예쁜' 등의 키워드를 보아 여성에 대한 토픽 모델링이 된 듯 하다.
- 7번 토픽은 인종, 국가에 대한 토픽 모델링으로 볼 수 있다.
- 그 외 나머지는 뚜렷한 경향성을 보이지 않는다. 불용어 추가나 파라미터 조정, 품사의 추가 및 제거가 필요해 보인다. 

# 시각화

In [ ]:
import pyLDAvis as vis

lda_vis_data = ctm.get_ldavis_data_format(vocab, training_dataset, n_samples=10)

ctm_pd = vis.prepare(**lda_vis_data)
vis.display(ctm_pd)
     

/usr/local/lib/python3.8/dist-packages/past/types/oldstr.py:5: DeprecationWarning: Using or importing the ABCs from 'collections' instead of from 'collections.abc' is deprecated since Python 3.3, and in 3.10 it will stop working
  from collections import Iterable
/usr/local/lib/python3.8/dist-packages/past/builtins/misc.py:4: DeprecationWarning: Using or importing the ABCs from 'collections' instead of from 'collections.abc' is deprecated since Python 3.3, and in 3.10 it will stop working
  from collections import Mapping
Sampling: [10/10]: : 10it [00:14,  1.45s/it]
/usr/local/lib/python3.8/dist-packages/pyLDAvis/_prepare.py:246: FutureWarning: In a future version of pandas all arguments of DataFrame.drop except for the argument 'labels' will be keyword-only
  default_term_info = default_term_info.sort_values(
